In [1]:
# pip install gymnasium
# pip install gym
# pip install swig

使用強化學習方法來尋找 0050.TW 這檔標的的買賣時間點。\
初始資金設置為 20000 元。\
訓練集數據範圍為 2020/1/1 至 2020/12/31，測試集數據範圍為 2021/1/1 至 2021/12/31。

### 載入所需套件

In [ ]:
import random

import gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import yfinance as yf
from stable_baselines3 import A2C, DQN, PPO

### 使用 gym 建置環境的架構

In [3]:
class StockTradingEnv(gym.Env):
    def __init__(self):
        """
        初始化環境，設定初始參數。
        這個方法主要負責初始化環境中的 State 或是 Action 等。
        """
        pass

    def reset(self):
        """重置環境，將所有狀態重置為初始值。這個方法通常在每一個新的 Episode 開始時被調用。"""
        pass

    def step(self, action):
        """
        根據執行的 Action 來更新環境的 State，並計算 Reward。
        同時也會檢查當前的 Episode 是否結束。
        """
        pass

    def render(self, mode="human"):
        """顯示資訊。"""
        pass

### init 方法範例


In [ ]:
observation_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
print(observation_space)

In [ ]:
observation_space = gym.spaces.Box(low=0, high=255, shape=(64, 64, 3))
print(observation_space)

In [ ]:
action_space = gym.spaces.Discrete(n=3)
print(action_space)

In [7]:
class StockTradingEnv(gym.Env):
    def __init__(self):
        """
        初始化環境，設定初始參數。
        這個方法主要負責初始化環境中的 State 或是 Action 等。
        """
        pass

    def reset(self):
        """重置環境，將所有狀態重置為初始值。這個方法通常在每一個新的 Episode 開始時被調用。"""
        pass

    def step(self, action):
        """
        根據執行的 Action 來更新環境的 State，並計算 Reward。
        同時也會檢查當前的 Episode 是否結束。
        """
        pass

    def render(self, mode="human"):
        """顯示資訊。"""
        pass

### reset 方法範例

In [8]:
class StockTradingEnv(gym.Env):
    def __init__(self):
        """
        初始化環境，設定初始參數。
        這個方法主要負責初始化環境中的 State 或是 Action 等。
        """
        pass

    def reset(self):
        """重置環境，將所有狀態重置為初始值。這個方法通常在每一個新的 Episode 開始時被調用。"""
        pass

    def step(self, action):
        """
        根據執行的 Action 來更新環境的 State，並計算 Reward。
        同時也會檢查當前的 Episode 是否結束。
        """
        pass

    def render(self, mode="human"):
        """顯示資訊。"""
        pass

### step 方法範例

In [9]:
class StockTradingEnv(gym.Env):
    def __init__(self):
        """
        初始化環境，設定初始參數。
        這個方法主要負責初始化環境中的 State 或是 Action 等。
        """
        pass

    def reset(self):
        """重置環境，將所有狀態重置為初始值。這個方法通常在每一個新的 Episode 開始時被調用。"""
        pass

    def step(self, action):
        """
        根據執行的 Action 來更新環境的 State，並計算 Reward。
        同時也會檢查當前的 Episode 是否結束。
        """
        pass

    def render(self, mode="human"):
        """顯示資訊。"""
        pass

### render 方法範例

In [10]:
class StockTradingEnv(gym.Env):
    def __init__(self):
        """
        初始化環境，設定初始參數。
        這個方法主要負責初始化環境中的 State 或是 Action 等。
        """
        pass

    def reset(self):
        """重置環境，將所有狀態重置為初始值。這個方法通常在每一個新的 episode 開始時被調用。"""
        pass

    def step(self, action):
        """
        根據執行的 Action 來更新環境的 State，並計算 Reward。
        同時也會檢查當前的 episode 是否結束。
        """
        pass

    def render(self, mode="human"):
        """顯示資訊。"""
        pass

### 先準備訓練資料和測試資料

In [ ]:
# 下載 0050.TW 的歷史資料，時間範圍為 2020-01-01 至 2020-12-31 當作訓練資料
train_df = (
    pd.DataFrame(yf.download("0050.TW", start="2020-01-01", end="2020-12-31"))
    .droplevel("Ticker", axis=1)
    .reset_index()
    .ffill()
)
train_df.index = train_df["Date"]
train_df.columns.name = None
train_df = train_df.drop(columns=["Date"])


# 下載 0050.TW 的歷史資料，時間範圍為 2021-01-01 至 2021-12-31 當作測試資料
test_df = (
    pd.DataFrame(yf.download("0050.TW", start="2021-01-01", end="2021-12-31"))
    .droplevel("Ticker", axis=1)
    .reset_index()
    .ffill()
)
test_df.index = test_df["Date"]
test_df.columns.name = None
test_df = test_df.drop(columns=["Date"])

### 環境建置

In [12]:
class StockTradingEnv(gym.Env):
    def __init__(self, df):
        """
        初始化股票交易環境。
        設定交易環境中的參數和狀態，包括初始資金、手續費、動作空間和觀察空間。
        """
        super(StockTradingEnv, self).__init__()

        self.df = df  # 股票的歷史數據
        self.stock_dim = 1  # 股票數量，這裡設定為1（如0050.TW）
        self.initial_amount = 20000  # 初始資金設定為 20000
        self.buy_cost_pct = 0.001425  # 買入股票時的手續費
        self.sell_cost_pct = 0.001425 + 0.003  # 賣出股票時的手續費和交易稅
        self.reward_scaling = 1e-2  # Reward 的縮放(正規化)比例

        # 定義 Action 一維空間，表示賣出和買入的動作
        # 數值 0 表示執行賣出動作
        # 數值 1 表示保持不變
        # 數值 2 表示執行買入動作
        self.action_space = gym.spaces.Discrete(3)

        # 定義 State 七維空間，State 包含現金、持有的股票數量、股票價格數據(開高低收量)
        # State 可能數值範圍從0到無限大
        # State[0]:現金； State[1]:持有的股票數量； State[2]:開盤價； State[3]:最高價；
        # State[4]:最低價； State[5]:收盤價； State[6]:交易量
        self.observation_space = gym.spaces.Box(
            low=0, high=np.inf, shape=(2 + len(self.df.columns),), dtype=np.float32
        )
        # 重置 Environment State 為初始 State
        self.reset()

    def reset(self):
        """
        重置環境狀態為初始狀態，通常在每個 Episode 的開始時調用。
        初始化現金、持有股票數量、第一天的股票價格等。
        """
        # 設置隨機種子
        random.seed(7777)
        np.random.seed(7777)
        torch.manual_seed(7777)

        self.day = 0  # 初始化執行天數為第一天
        # 取出第一天歷史資料當作初始股票價格數據
        self.data = self.df.iloc[self.day]

        # 初始化現金 State[0] 為初始金額 self.initial_amount
        # 初始化持有的股票數量 State[1] = 0
        # 初始化股票價格數據(開高低收量) State[2]~State[6] 是第一天歷史資料
        self.state = np.array(
            [self.initial_amount, 0] + self.data.tolist(), dtype=np.float32
        )

        # self.terminal 的值用來判斷交易是否應該結束，如果是 True 就是一個 Episode 結束
        # 重置終止標誌為 False，表示新的一輪交易開始
        self.terminal = False

        self.reward = 0  # 初始化累積收益值為0
        self.asset_memory = [self.initial_amount]  # 初始化資產記錄為初始資金
        self.actions_memory = []  # 初始化 Action 記錄
        self.date_memory = [self.data.name]  # 初始化日期記錄為第一天日期
        self.trade_memory = []  # 初始化交易記錄
        return self.state

    def step(self, action):
        """
        根據 Action 來更新環境狀態，計算回報，並判斷 Episode 是否結束。
        """
        # 檢查是否到達資料的最後一天，若到達最後一天，則設置 terminal 為 True
        self.terminal = self.day >= len(self.df.index) - 1
        if self.terminal:
            return self.state, self.reward, self.terminal, {}

        # 計算當前總資產(現金+股票的價值)
        begin_total_asset = self.state[0] + self.state[1] * self.state[2]

        if action == 0:  # 執行賣出操作
            # 賣出的股票數量是目前持有的股票數量
            sell_num_shares = self.state[1]
            # 計算賣出股票後獲得的現金(數量*開盤價)，並扣除賣出手續費
            sell_amount = sell_num_shares * self.state[2] * (1 - self.sell_cost_pct)
            self.state[0] += sell_amount  # 更新現金餘額
            self.state[1] -= sell_num_shares  # 更新持有的股票數量
            # 記錄賣出交易的日期、類型、數量和價格
            self.trade_memory.append(
                (self.data.name, "sell", sell_num_shares, self.state[2])
            )

        if action == 2:  # 執行買入操作
            # 計算能夠買入的股票數量，不能超過可用的現金數量(現金除以當下價格)
            buy_num_shares = self.state[0] // (self.state[2] * (1 + self.buy_cost_pct))
            # 計算買入股票需要的現金(數量*開盤價)，並加上買入手續費
            buy_amount = buy_num_shares * self.state[2] * (1 + self.buy_cost_pct)
            self.state[0] -= buy_amount  # 更新現金餘額
            self.state[1] += buy_num_shares  # 更新持有的股票數量
            # 記錄買入交易的日期、類型、數量和價格
            self.trade_memory.append(
                (self.data.name, "buy", buy_num_shares, self.state[2])
            )

        self.day += 1  # 更新天數到下一天
        self.data = self.df.iloc[self.day]  # 取得下一天的數據
        # 更新 State，包括目前現金、目前持有的股票數量和下一天股票價格數據
        self.state = np.array(
            [self.state[0], self.state[1]] + self.data.tolist(), dtype=np.float32
        )

        # 計算新的總資產
        end_total_asset = self.state[0] + self.state[1] * self.state[2]

        # 計算 Reward 為總資產的變化，並將 Reward 進行正規化動作
        self.reward = end_total_asset - begin_total_asset
        self.reward = self.reward * self.reward_scaling

        # 根據不同的動作給予額外的小額獎勵
        if action == 0:  # 賣出獲得額外獎勵
            self.reward += 0.05
        elif action == 1:  # 保持獲得小額獎勵
            self.reward += 0.005

        # 記錄資產、日期和 Action 的變化
        self.asset_memory.append(end_total_asset)
        self.date_memory.append(self.data.name)
        self.actions_memory.append(action)

        # 回傳當下 State、Reward、是否終止和其他額外訊息
        return self.state, self.reward, self.terminal, {}

    def render(self, mode="human"):
        """顯示交易環境的當前狀態。"""
        print(f"目前日期: {self.data.name}")
        print(f"目前執行到第幾天: {self.day}")
        print(f"目前Reward: {self.reward}")
        print(f"目前現金金額: {self.state[0]}")
        print(f"目前持有股票數量: {self.state[1]}")
        print(f"目前股票開盤價: {self.state[2]}")
        print(f"目前總資產: {self.state[0] + self.state[1] * self.state[2]}")

    def save_asset_memory(self):
        """儲存每一天的總資產記錄，並回傳一個包含日期和資產的 DataFrame。"""
        return pd.DataFrame({"date": self.date_memory, "asset": self.asset_memory})

    def save_action_memory(self):
        """儲存每一天的 Action 記錄，並回傳一個包含日期和動作的 DataFrame。"""
        return pd.DataFrame(
            {"date": self.date_memory[:-1], "actions": self.actions_memory}
        )

    def save_trade_memory(self):
        """儲存發生交易的記錄，並回傳一個包含日期、交易類型、交易股票數量和交易價格的 DataFrame。"""
        return pd.DataFrame(
            self.trade_memory, columns=["date", "type", "shares", "price"]
        )

### Policy Based Method
- Proximal Policy Optimization(PPO)：適用於連續和離散動作空間

In [ ]:
# 使用訓練數據集 train_df 創建交易環境
env = StockTradingEnv(train_df)

# 使用 PPO 算法創建模型
# 使用多層感知機（Multi-Layer Perceptron, MLP）作為策略和價值網絡的結構
model = PPO("MlpPolicy", env, verbose=1)

# 訓練模型，總訓練步數為 30000 步
model.learn(total_timesteps=30000)

# 將訓練好的模型儲存為文件 PPO_0050.zip
model.save("PPO_0050")

# 載入儲存的模型
model = PPO.load("PPO_0050")

# 評估模型在訓練集上的表現
obs = env.reset()
for i in range(len(train_df)):
    action, _states = model.predict(obs)  # 使用模型預測動作
    obs, rewards, done, info = env.step(action)  # 執行動作
    env.render()  # 顯示環境狀態

# 儲存交易結果
asset_memory = env.save_asset_memory()  # 儲存資產變化記錄
asset_memory.to_csv("PPO_assets.csv")

action_memory = env.save_action_memory()  # 儲存 Action 記錄
action_memory.to_csv("PPO_actions.csv")

trade_memory = env.save_trade_memory()  # 儲存交易記錄
trade_memory.to_csv("PPO_trades.csv")

# 畫出資產變化圖
plt.plot(asset_memory["date"], asset_memory["asset"])
plt.xlabel("Date")
plt.ylabel("Asset Value")
plt.title("Cumulative Asset Value over Time (PPO)")
plt.xticks(rotation=45)
plt.show()

# 顯示交易記錄
# 顯示買入進場的時間點
buy_df = trade_memory[(trade_memory["type"] == "buy")]
buy_df = buy_df[(trade_memory["shares"] != 0)]
print("買入進場的時間點:")
print(buy_df)

# 顯示賣出出場的時間點
sell_df = trade_memory[(trade_memory["type"] == "sell")]
sell_df = sell_df[(trade_memory["shares"] != 0)]
print("賣出出場的時間點:")
print(sell_df)

### Actor Critic Method
- Advantage Actor-Critic(A2C)：適用於連續和離散動作空間

In [ ]:
# 使用訓練數據集 train_df 創建交易環境
env = StockTradingEnv(train_df)

# 使用 A2C 算法創建模型
model = A2C("MlpPolicy", env, verbose=1)

# 訓練模型，總訓練步數為 30000 步
model.learn(total_timesteps=30000)

# 將訓練好的模型儲存為文件 A2C_0050.zip
model.save("A2C_0050")

# 載入儲存的模型
model = A2C.load("A2C_0050")

# 評估模型在訓練集上的表現
obs = env.reset()
for i in range(len(train_df)):
    action, _states = model.predict(obs)  # 使用模型預測動作
    obs, rewards, done, info = env.step(action)  # 執行動作
    env.render()  # 顯示環境狀態

# 儲存交易結果
asset_memory = env.save_asset_memory()  # 儲存資產變化記錄
asset_memory.to_csv("A2C_0050_assets.csv")

action_memory = env.save_action_memory()  # 儲存 Action 記錄
action_memory.to_csv("A2C_0050_actions.csv")

trade_memory = env.save_trade_memory()  # 儲存交易記錄
trade_memory.to_csv("A2C_0050_trades.csv")

# 畫出資產變化圖
plt.plot(asset_memory["date"], asset_memory["asset"])
plt.xlabel("Date")
plt.ylabel("Asset Value")
plt.title("Cumulative Asset Value over Time (A2C)")
plt.xticks(rotation=45)
plt.show()

# 顯示交易記錄
# 顯示買入進場的時間點
buy_df = trade_memory[(trade_memory["type"] == "buy")]
buy_df = buy_df[(trade_memory["shares"] != 0)]
print("買入進場的時間點:")
print(buy_df)

# 顯示賣出出場的時間點
sell_df = trade_memory[(trade_memory["type"] == "sell")]
sell_df = sell_df[(trade_memory["shares"] != 0)]
print("賣出出場的時間點:")
print(sell_df)

### Valued Based Method
- Deep Q-Network(DQN)：適用於離散動作空間

In [ ]:
# 使用訓練數據集 train_df 創建交易環境
env = StockTradingEnv(train_df)

# 使用 DQN 算法創建模型
model = DQN(policy="MlpPolicy", env=env, verbose=1)

# 訓練模型，總訓練步數為 1000 步
model.learn(total_timesteps=1000)

# 將訓練好的模型儲存為文件 DQN_0050.zip
model.save("DQN_0050")

# 載入儲存的模型
model = DQN.load("DQN_0050")

# 評估模型在訓練集上的表現
obs = env.reset()
for i in range(len(train_df)):
    action, _states = model.predict(obs)  # 使用模型預測動作
    obs, rewards, done, info = env.step(action)  # 執行動作
    env.render()  # 顯示環境狀態

# 儲存交易結果
asset_memory = env.save_asset_memory()  # 儲存資產變化記錄
asset_memory.to_csv("DQN_0050_assets.csv")

action_memory = env.save_action_memory()  # 儲存 Action 記錄
action_memory.to_csv("DQN_0050_actions.csv")

trade_memory = env.save_trade_memory()  # 儲存交易記錄
trade_memory.to_csv("DQN_0050_trades.csv")

# 畫出資產變化圖
plt.plot(asset_memory["date"], asset_memory["asset"])
plt.xlabel("Date")
plt.ylabel("Asset Value")
plt.title("Cumulative Asset Value over Time (DQN)")
plt.xticks(rotation=45)
plt.show()

# 顯示交易記錄
# 顯示買入進場的時間點
buy_df = trade_memory[(trade_memory["type"] == "buy")]
buy_df = buy_df[(trade_memory["shares"] != 0)]
print("買入進場的時間點:")
print(buy_df)

# 顯示賣出出場的時間點
sell_df = trade_memory[(trade_memory["type"] == "sell")]
sell_df = sell_df[(trade_memory["shares"] != 0)]
print("賣出出場的時間點:")
print(sell_df)